In [ ]:
# Estratégia de Pair Trading com GLD e IAU (ETFs de ouro)
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import coint
from scipy.stats import zscore

In [ ]:
# 1. Coleta de dados - 5 anos de histórico
tickers = ['GLD', 'IAU']
start = '2019-07-01'
end = '2024-07-01'
df = yf.download(tickers, start=start, end=end)['Adj Close'].dropna()

C:\Users\Usuário\AppData\Local\Temp\ipykernel_10072\2731993309.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(tickers, start=start, end=end)['Close'].dropna()
[*********************100%***********************]  2 of 2 completed


In [ ]:
# 2. Teste de cointegração (Engle-Granger)
score, pvalue, _ = coint(df['GLD'], df['IAU'])
print(f"P-valor da cointegração: {pvalue:.4f}")
if pvalue < 0.05:
    print("Os ativos são cointegrados — válido para Pair Trading")
else:
    print(" Os ativos não são cointegrados — cuidado!")

P-valor do teste de cointegração: 0.2878
Os ativos não são cointegrados — atenção!


In [ ]:
# 3. Calcular spread e Z-score
spread = df['GLD'] - df['IAU']
zscore_spread = zscore(spread)

In [ ]:
# 4. Estratégia de negociação (baseada no Z-score do spread)
entry_z = 1.0     # Entrada quando zscore > 1 ou < -1
exit_z = 0.0      # Saída quando zscore ≈ 0

signals = pd.DataFrame(index=df.index)
signals['Z-Score'] = zscore_spread
signals['Position_GLD'] = 0
signals['Position_IAU'] = 0

In [ ]:
# Sinais de entrada e saída
signals.loc[signals['Z-Score'] > entry_z, ['Position_GLD', 'Position_IAU']] = [-1, 1]  # Vende GLD, compra IAU
signals.loc[signals['Z-Score'] < -entry_z, ['Position_GLD', 'Position_IAU']] = [1, -1]  # Compra GLD, vende IAU
signals.loc[abs(signals['Z-Score']) < exit_z, ['Position_GLD', 'Position_IAU']] = [0, 0]  # Zera posições

# Preencher posições ao longo do tempo
signals.fillna(method='ffill', inplace=True)

In [ ]:
# 5. Cálculo dos retornos da estratégia
returns = (signals[['Position_GLD', 'Position_IAU']].shift(1) * df.pct_change()).sum(axis=1)
cumulative_returns = (1 + returns).cumprod()

# 6. Métricas de desempenho
sharpe = np.mean(returns) / np.std(returns) * np.sqrt(252)
running_max = cumulative_returns.cummax()
drawdown = (cumulative_returns - running_max) / running_max
max_drawdown = drawdown.min()


C:\Users\Usuário\AppData\Local\Temp\ipykernel_10072\146003827.py:6: RuntimeWarning: invalid value encountered in scalar divide
  sharpe = np.mean(returns) / np.std(returns) * np.sqrt(252)


In [ ]:
# 7. Gráficos

plt.figure(figsize=(14, 8))

# Preço dos ativos
plt.subplot(3, 1, 1)
plt.plot(df['GLD'], label='GLD')
plt.plot(df['IAU'], label='IAU')
plt.title('Preços Históricos: GLD e IAU (5 anos)')
plt.legend()

# Z-score do spread
plt.subplot(3, 1, 2)
plt.plot(signals['Z-Score'], color='purple')
plt.axhline(entry_z, color='red', linestyle='--')
plt.axhline(-entry_z, color='green', linestyle='--')
plt.axhline(0, color='black', linestyle='-')
plt.title('Z-Score do Spread (GLD - IAU)')

# Curva de capital
plt.subplot(3, 1, 3)
plt.plot(cumulative_returns, label='Curva de Capital', color='darkblue')
plt.title(f'Rendimento da Estratégia | Sharpe: {sharpe:.2f} | Max Drawdown: {max_drawdown:.2%}')
plt.legend()

plt.tight_layout()
plt.show()